This notebook is starting point for Home Credit Default Risk Kaggle Problem Solution
Primary objective of this notbook is to load data from /input.nosync folder and perform basic EDA.
Detailed modelling and prediction should be done in separate notebook based on EDA results that we get here

In [41]:
# all the imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import pprint

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer

In [4]:
#Notebook lavel configurations
warnings.filterwarnings('ignore')
pp = pprint.PrettyPrinter(indent=4)

In [34]:
# List input files, if not available please download from below url inot a /input.nosync folder
input_dir = 'input.nosync'
input_files = os.listdir(input_dir)
if input_files is None or len(input_files) < 11 :
    raise Exception('You do not have all the files in {} directory'.format(input_dir))

print('You have all the input files listed below')
pp.pprint(input_files)

You have all the input files listed below
[   'application_tes',
    'application_tes.zip',
    'application_test.csv',
    'application_train',
    'application_train.csv',
    'bureau.csv.zip',
    'bureau_balance.csv.zip',
    'credit_card_balance.csv.zip',
    'HomeCredit_columns_description.csv',
    'installments_payments',
    'installments_payments.csv',
    'installments_payments.csv.zip',
    'POS_CASH_balance.csv.zip',
    'previous_application.csv.zip',
    'sample_submission.csv.zip']


Read in application_train and application test, note any changes to train need to be made to test as well

In [35]:
# Read application_train.csv
app_train = pd.read_csv(input_dir+'/application_train.csv')
app_test = pd.read_csv(input_dir+'/application_test.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Convert categorical columns to numeric using One Hot Encoding

In [36]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 246)
Testing Features shape:  (48744, 242)


Align train and test so clumns match

In [38]:
train_labels = app_train['TARGET']
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
app_train['TARGET'] = train_labels

In [42]:
print(app_test.shape)
print(app_train.shape)

(48744, 242)
(307511, 243)


Impute missing values, using the column mean

In [57]:
imputer = Imputer()
app_train.drop('TARGET', axis=1)
app_train_filled = imputer.fit_transform(app_train)
app_test_filled = imputer.fit_transform(app_test)
